In [1]:
import os
print(os.getcwd())

/scratch/midway3/aesteva/fea_project


In [2]:
# This Notebook allows you to train both a cross-encoder and a bi-encoder model on your own dataset.
# Don't run this unless you have sufficient computational resources. Please.

## Install Libraries

In [3]:
# # Install necessary libraries for Midway3 Cluster
# import sys
# import os
# import shutil

# username = os.environ.get('USER')
# scratch_base = f"/scratch/midway3/{username}"
# custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
# py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
# site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")


# print(f"Targeting: {site_packages}")

# #
# # We delete the entire library folder to ensure NO conflicting files remain.
# if os.path.exists(custom_lib_path):
#     print(f"Removing old libraries at {custom_lib_path}...")
#     try:
#         shutil.rmtree(custom_lib_path)
#         print("Cleanup complete. Folder is empty.")
#     except Exception as e:
#         print(f"Warning: Could not fully delete folder ({e}). Proceeding anyway...")
# else:
#     print("Folder was already clean.")


# install_cmd = (
#     f"pip install --target='{site_packages}' "
#     "--upgrade "
#     "--no-cache-dir "          # Force download of fresh, uncorrupted files
#     "'numpy<2.0' "              # Prevent Numpy 2.0 crashes
#     "'pandas' "
#     "'bottleneck' "             # Fixes Pandas warning
#     "'numexpr' "                # Fixes Pandas warning
#     "'torch' "                  # Install Torch in scratch
#     "'torchvision' "            # Install Torchvision in scratch (matches Torch)
#     "'accelerate>=0.31.0' "
#     "'transformers' "
#     "'sentence-transformers' "
#     "'datasets' "
#     "'botocore' "
#     "'aiobotocore' "
#     "'s3fs' "
#     "'openpyxl' "
#     "'sentencepiece' "
# )

# print("\nStarting Clean Install (This may take a few minutes)...")
# exit_code = os.system(install_cmd)

# if exit_code == 0:
#     print("\nSUCCESS: Complete stack installed.")
# else:
#     print(f"\nERROR: Installation failed with code {exit_code}")

# print("\n##### RESTART KERNEL NOW #####")

Targeting: /scratch/midway3/aesteva/my_custom_libs/lib/python3.11/site-packages
Removing old libraries at /scratch/midway3/aesteva/my_custom_libs...
Cleanup complete. Folder is empty.

Starting Clean Install (This may take a few minutes)...
  Obtaining dependency information for numpy<2.0 from https://files.pythonhosted.org/packages/3a/d0/edc009c27b406c4f9cbc79274d6e46d634d139075492ad055e3d68445925/numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.7 MB/s eta 0:00:00
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/bf/c9/63f8d545568d9ab91476b1818b4741f521646cbdd151c6efebf40d6de6f7/pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 18.9 MB/s eta 0:00:00
  Obtaining dependency information for bottleneck from https://files.pythonhosted.org/packages/16/f4/4fcbebcbc42376a

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-core 0.2.9 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires nbformat==5.9.2, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.75 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0.3 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.5 which is incompatible.
langchainhub 0.1.20 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
tables 3.8.0 requires blosc2~=2.0.0, but yo


SUCCESS: Complete stack installed.

##### RESTART KERNEL NOW #####


In [1]:
import sys
import os

# --- 1. SETUP PATHS FIRST ---
username = os.environ.get('USER')
scratch_base = f"/scratch/midway3/{username}"
custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
hf_cache_path = os.path.join(scratch_base, "hf_cache")

py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")

# FORCE custom path to the FRONT of the list
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

print(f"Python is looking here first: {sys.path[0]}")

# --- 2. ENV VARIABLES ---
os.environ['HF_HOME'] = hf_cache_path
os.environ['TRANSFORMERS_CACHE'] = hf_cache_path
os.environ['PYTHONUSERBASE'] = custom_lib_path

# --- 3. NOW IMPORT LIBRARIES ---
# Only import AFTER sys.path is updated
import datasets 
import sentence_transformers
from sentence_transformers import SentenceTransformer

# --- 4. VERIFY PATHS ---
print(f"Datasets is loaded from: {os.path.dirname(datasets.__file__)}")

if "software" in os.path.dirname(datasets.__file__):
    print("FAILURE: Still loading system datasets.")
else:
    print("SUCCESS: All libraries isolated in scratch.")

Python is looking here first: /scratch/midway3/aesteva/my_custom_libs/lib/python3.11/site-packages


/scratch/midway3/aesteva/my_custom_libs/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Datasets is loaded from: /scratch/midway3/aesteva/my_custom_libs/lib/python3.11/site-packages/datasets
SUCCESS: All libraries isolated in scratch.


## Setup

In [2]:
import importlib
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from typing import List, Tuple, Any, Mapping, Iterable, Dict, Literal

import free_entailments_algorithm_utils as fea


import sys
import os
import transformers.utils.hub
import transformers.tokenization_utils_base
from sentence_transformers import CrossEncoder

In [3]:
importlib.reload(fea)

<module 'free_entailments_algorithm_utils' from '/scratch/midway3/aesteva/fea_project/free_entailments_algorithm_utils.py'>

In [4]:
df_p = pd.read_excel('/home/aesteva/Downloads/Dropbox(1)/ClauseLevel_df_p.xlsx') #premises
df_sc = pd.read_excel("/home/aesteva/Downloads/Dropbox(1)/ClauseLevel_df_sc.xlsx") #conclusions
df_llm = pd.read_csv("/home/aesteva/Downloads/Dropbox(1)/all_results (1).csv") #llm results

In [5]:
# Unify clauses form premises and conclusions into a single dataframe
df_clause = pd.concat([df_p,df_sc])

In [6]:
# Merge clause pairs with entailment and text
df_clause_entailment = fea.merge_pairwise_texts(
    df1 = df_clause,
    df2 = df_llm,
    df1_cols = ['sentence_id', 'sentence'],
    df2_cols = ['sentence_id_1', 'sentence_id_2', 'twoway']
)

In [7]:
print(f"Shape: {df_clause_entailment.shape}")
df_clause_entailment.head()

Shape: (448369, 5)


,id1,id2,text1,text2,verdict
0,B0860003sc,S6976003sc,The king's maintenance and support must ensure...,The King should retain the authority to ensure...,NO
1,B1135001sc,S10714001sc,Effective monarchy governance requires laws to...,Rejecting the proposed law is essential to mai...,NO
2,B0216001sc,S6283001sc,A king's authority is shaped by legal frameworks,The King has the authority to override specifi...,NO
3,B0249004sc,S11150001sc,The rights of the people,Preserving individual rights,YES
4,B0859002sc,S8937002sc,A king must maintain his rights to protect his...,The King must uphold the rule of law,YES


# Cleaning up Cache

In [1]:
## RUN IF YOU ARE ON MIDWAY3 TO CLEAR GPU CACHE ##
 
import torch
import gc

# # 1. Force Python's garbage collector to release references
gc.collect()

# # 2. Force PyTorch to release cached CUDA memory
torch.cuda.empty_cache()

print("GPU Cache Cleared.")

GPU Cache Cleared.


# Training bi-encoder

In [9]:
#The commented specifications are for RCC Midway3 use only. I do not recommend these for your computer. Ask an LLM what the best specs for you are.

fea.train_bert_model(
    df_train=df_clause_entailment,
    text_col1='text1',
    text_col2='text2',
    verdict_col='verdict',
    positive_label='YES',
    base_model_name='BAAI/bge-large-en-v1.5',
    output_path="./fine_tuned_bi_model",
    batch_size=128,  # <--- INCREASED for A100
    num_epochs=1,    # <--- DECREASED to save time (usually enough)
    use_amp=True,    # Explicitly confirming FP16 for this computer
    num_workers=16    # Explicitly using your CPU cores
)


Training bi-encoder on 448369 labeled pairs from BAAI/bge-large-en-v1.5...
Starting Bi-Encoder training with AMP=True...


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.012100
1000,0.009100
1500,0.008500
2000,0.008200
2500,0.007800
3000,0.007600
3500,0.007500


Saving fine-tuned model to ./fine_tuned_bi_model...
Training complete and VRAM cleared.


# Training cross-encoder

In [10]:

def _safe_list_templates(*args, **kwargs):
    return []

transformers.utils.hub.list_repo_templates = _safe_list_templates
transformers.tokenization_utils_base.list_repo_templates = _safe_list_templates
print("1. [Success] 404 Error blocked.")


if not hasattr(CrossEncoder, "_original_init"):
    CrossEncoder._original_init = CrossEncoder.__init__

def _patched_init(self, *args, **kwargs):
    if "model_kwargs" not in kwargs:
        kwargs["model_kwargs"] = {}
    # Force the model to ignore the size mismatch in the final layer
    kwargs["model_kwargs"]["ignore_mismatched_sizes"] = True
    # Call the original function
    CrossEncoder._original_init(self, *args, **kwargs)

CrossEncoder.__init__ = _patched_init
print("2. [Success] Shape mismatch patch applied.")


try:
    import sentencepiece
    print("3. [Success] SentencePiece is installed and detected.")
except ImportError:
    print("3. [ERROR] SentencePiece is STILL missing. Did you restart the kernel?")

print("\nREADY: You can now run your training code.")

1. [Success] 404 Error blocked.
2. [Success] Shape mismatch patch applied.
3. [Success] SentencePiece is installed and detected.

READY: You can now run your training code.


In [12]:
#The commented specifications are for RCC Midway3 use only. I do not recommend these for your computer. Ask an LLM what the best specs for you are.

fea.train_bert_model(
    df_train=df_clause_entailment,
    text_col1='text1',
    text_col2='text2',
    verdict_col='verdict',
    base_model_name="cross-encoder/nli-deberta-v3-base",
    output_path="./fine_tuned_nli_model",
    model_type="cross-encoder", 
    batch_size=128,  # <--- INCREASED for A100
    num_epochs=1,    # <--- DECREASED to save time (usually enough)
    use_amp=True,    # Explicitly confirming FP16
    num_workers=16    # Explicitly using your CPU cores
)

Training cross-encoder on 448369 labeled pairs from cross-encoder/nli-deberta-v3-base...


The CrossEncoder `tokenizer_args` argument was renamed and is now deprecated, please use `tokenizer_kwargs` instead.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-v3-base and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting Cross-Encoder training with AMP=True...


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Step,Training Loss
500,0.305300
1000,0.224100
1500,0.205300
2000,0.195500
2500,0.187200
3000,0.179100
3500,0.176800


Saving fine-tuned model to ./fine_tuned_nli_model...
Training complete and VRAM cleared.
